# 06 Calculate Distance
- input: `data/05_augment_thai.csv`

In [1]:
import os
import pickle
from functools import lru_cache

import pandas as pd
from utils import clean_text
from tqdm.notebook import tqdm
from scipy.spatial.distance import cosine

tqdm.pandas()

In [2]:
data = pd.read_csv('data/05_augment_thai.csv')
data

,question,answers,source,id,en_aug,th_aug,context,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,...,th_qcpg_0.2_aug,th_qcpg_0.5_aug,th_qcpg_0.8_aug,th_qcpg_0.2_llm_gec_aug,th_qcpg_0.5_llm_gec_aug,th_qcpg_0.8_llm_gec_aug,th_wordnet_aug,th_thai2fit_aug,th_ltw2v_aug,th_fasttext_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,...,วันเดือนปีเกิดของภูเก็ตข่านจูเนียร์คืออะไร?,วันเกิดของภูเก็ตข่านคืออะไร?,วันเดือนปีเกิดของ Phattin Sarutin คืออะไร?,วันเกิดของภูเก็ตข่านคืออะไร?,phatthira sarutpah ng คืออะไร,วันเดือนปีเกิดของ Phatthira Sarutin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันเท่าไร ?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ เกิดวันที่เท่าไ...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่่ ?
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,...,คุณกำลังเล่นอะไรเมื่อ Sarutina กำลังเล่นอยู่?,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutina?,คุณเล่นอะไรกับ Phatthira Sarutong?,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutin?,คุณกำลังเล่นอะไรเมื่อคุณเป็น phatthira sarut,คุณกำลังเล่นอะไรใน Phatthira Sarutong pH,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร ?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร??,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,...,อาชีพนักฟิสิกส์คืออะไร?,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร,อาชีพของ phatthira teeratha phi คืออะไร,อาชีพนักฟิสิกส์คืออะไร?,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร,อาชีพของ phatthira teeratha phi คืออะไร,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร ?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร??,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4 ?
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,...,Phatthira Sarutpah Phoenix จบการศึกษา,phatthira sarutpah pH ประเทศอะไร,Phatthira Sarutpong Phokin จบการศึกษาจาก?,phatthira sarutpah pH ปีอะไร,phatthira sarutpah pH ปีอะไร,Phatthira Sarutpong Phokin ประเทศอะไร,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการอบรมจากประเทศอะไร ?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ จบการศึกษาจากปร...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,Who is Cleopatra's father?,พ่อของคลีโอพัตราคือใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,Who is the father of Cleopatra?,Who is Cleopatra's bi

In [3]:
# Clean all question columns
cols = [col for col in data.columns if col.startswith('th_')]
for col in tqdm(cols):
    data[col] = data[col].progress_apply(lambda x: clean_text(x, is_question=True))

data

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

/Users/parinzee/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/utils.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


  0%|          | 0/16980 [00:00<?, ?it/s]

/Users/parinzee/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/utils.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


...

  0%|          | 0/16980 [00:00<?, ?it/s]

/Users/parinzee/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/utils.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


  0%|          | 0/16980 [00:00<?, ?it/s]

/Users/parinzee/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/utils.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


,question,answers,source,id,en_aug,th_aug,context,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,...,th_qcpg_0.2_aug,th_qcpg_0.5_aug,th_qcpg_0.8_aug,th_qcpg_0.2_llm_gec_aug,th_qcpg_0.5_llm_gec_aug,th_qcpg_0.8_llm_gec_aug,th_wordnet_aug,th_thai2fit_aug,th_ltw2v_aug,th_fasttext_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,...,วันเดือนปีเกิดของภูเก็ตข่านจูเนียร์คืออะไร?,วันเกิดของภูเก็ตข่านคืออะไร?,วันเดือนปีเกิดของ Phattin Sarutin คืออะไร?,วันเกิดของภูเก็ตข่านคืออะไร?,phatthira sarutpah ng คืออะไร?,วันเดือนปีเกิดของ Phatthira Sarutin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันเท่าไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ เกิดวันที่เท่าไ...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,...,คุณกำลังเล่นอะไรเมื่อ Sarutina กำลังเล่นอยู่?,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutina?,คุณเล่นอะไรกับ Phatthira Sarutong?,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutin?,คุณกำลังเล่นอะไรเมื่อคุณเป็น phatthira sarut?,คุณกำลังเล่นอะไรใน Phatthira Sarutong pH?,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,...,อาชีพนักฟิสิกส์คืออะไร?,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร?,อาชีพของ phatthira teeratha phi คืออะไร?,อาชีพนักฟิสิกส์คืออะไร?,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร?,อาชีพของ phatthira teeratha phi คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,...,Phatthira Sarutpah Phoenix จบการศึกษา?,phatthira sarutpah pH ประเทศอะไร?,Phatthira Sarutpong Phokin จบการศึกษาจาก?,phatthira sarutpah pH ปีอะไร?,phatthira sarutpah pH ปีอะไร?,Phatthira Sarutpong Phokin ประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการอบรมจากประเทศอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ จบการศึกษาจากปร...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,Who is Cleopatra's father?,พ่อของคลีโอพัตราคือใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,Who is the father of Cleopatra?,Who is Cleopatra's

In [4]:
# Sanity check for nan
data.isna().sum()

question                    0
answers                     0
source                      0
id                          0
en_aug                      0
th_aug                      0
context                    34
en_llm_gec_aug              0
en_llm_paraphrase_aug       0
en_qcpg_0.2_aug             1
en_qcpg_0.5_aug             1
en_qcpg_0.8_aug             3
en_qcpg_0.2_llm_gec_aug     0
en_qcpg_0.5_llm_gec_aug     0
en_qcpg_0.8_llm_gec_aug     0
th_llm_gec_aug              0
th_llm_paraphrase_aug       0
th_qcpg_0.2_aug             0
th_qcpg_0.5_aug             0
th_qcpg_0.8_aug             0
th_qcpg_0.2_llm_gec_aug     0
th_qcpg_0.5_llm_gec_aug     0
th_qcpg_0.8_llm_gec_aug     0
th_wordnet_aug              0
th_thai2fit_aug             0
th_ltw2v_aug                0
th_fasttext_aug             0
dtype: int64

In [5]:
# The empty context is a result of google sheets quirks, so we will replace it with the one from step 01
# We can see that there are 34 missing values in the `context` column. We can fill it up with the original context from the first file by removing and merging.
original = pd.read_csv('data/01_prepare_dataset.csv')
data = data.drop(columns=["context"])
data = data.merge(original[["id", "context"]], on="id")
data

,question,answers,source,id,en_aug,th_aug,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,en_qcpg_0.5_aug,...,th_qcpg_0.5_aug,th_qcpg_0.8_aug,th_qcpg_0.2_llm_gec_aug,th_qcpg_0.5_llm_gec_aug,th_qcpg_0.8_llm_gec_aug,th_wordnet_aug,th_thai2fit_aug,th_ltw2v_aug,th_fasttext_aug,context
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,What is the birth date of phuket khan?,...,วันเกิดของภูเก็ตข่านคืออะไร?,วันเดือนปีเกิดของ Phattin Sarutin คืออะไร?,วันเกิดของภูเก็ตข่านคืออะไร?,phatthira sarutpah ng คืออะไร?,วันเดือนปีเกิดของ Phatthira Sarutin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันเท่าไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ เกิดวันที่เท่าไ...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,What are you playing when you are sarutina?,...,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutina?,คุณเล่นอะไรกับ Phatthira Sarutong?,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutin?,คุณกำลังเล่นอะไรเมื่อคุณเป็น phatthira sarut?,คุณกำลังเล่นอะไรใน Phatthira Sarutong pH?,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,What career path did phatthira teeratha phikin,...,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร?,อาชีพของ phatthira teeratha phi คืออะไร?,อาชีพนักฟิสิกส์คืออะไร?,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร?,อาชีพของ phatthira teeratha phi คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,What country did phatthira sarutpah ph,...,phatthira sarutpah pH ประเทศอะไร?,Phatthira Sarutpong Phokin จบการศึกษาจาก?,phatthira sarutpah pH ปีอะไร?,phatthira sarutpah pH ปีอะไร?,Phatthira Sarutpong Phokin ประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการอบรมจากประเทศอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ จบการศึกษาจากปร...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,Who is Cleopatra's father?,พ่อของคลีโอพัตราคือใคร?,Who is the father of Cleopatra?,Who is Cleopatra's biological father?,What was cleop

In [6]:
# Sort all the columns properly
all_cols = data.columns

sorted_cols = ['id', 'question', 'context', 'answers', 'source'] + sorted([col for col in all_cols if col.startswith('th_')]) + sorted([col for col in all_cols if col.startswith('en_')])
assert len(all_cols) == len(sorted_cols)

sorted_cols

['id',
 'question',
 'context',
 'answers',
 'source',
 'th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_ltw2v_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_thai2fit_aug',
 'th_wordnet_aug',
 'en_aug',
 'en_llm_gec_aug',
 'en_llm_paraphrase_aug',
 'en_qcpg_0.2_aug',
 'en_qcpg_0.2_llm_gec_aug',
 'en_qcpg_0.5_aug',
 'en_qcpg_0.5_llm_gec_aug',
 'en_qcpg_0.8_aug',
 'en_qcpg_0.8_llm_gec_aug']

In [7]:
data = data[sorted_cols]
data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,th_wordnet_aug,en_aug,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,en_qcpg_0.2_llm_gec_aug,en_qcpg_0.5_aug,en_qcpg_0.5_llm_gec_aug,en_qcpg_0.8_aug,en_qcpg_0.8_llm_gec_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันเท่าไร?,Phatthira Sarutpong Phokin What is the date of...,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,What is the birth date of phuket khan?,What is the birth date of phuket khan?,What is phatthira sarutpah ng date of,What is the date of birth of phattin sarutin?,What is the date of birth of phatthira sarutin?
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,Phatthira Sarutpong Phokin What are you playing?,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,What are you playing when you are a sarutin?,What are you playing when you are sarutina?,What are you playing when you are a phatthira ...,What are you playing with phatthira sarutong?,What are you playing in phatthira sarutong ph
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,What is the career of Phatthira Teerathiyapong...,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,What is the career of a physicist?,What career path did phatthira teeratha phikin,What career path did phatthira teeratha phikin,What is the career of phatthira teeratha phi,What is the career of phatthira teeratha phi
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการอบรมจากประเทศอะไร?,Phatthira Sarutpong Phokin Graduated from what...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,What year did phatthira sarutpah ph,What country did phatthira sarutpah ph,What year did phatthira sarutpah ph,Phatthira sarutpong phokin graduated from?,What country did phatthira sarutpong phokin
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρ

# Use mUSE to calculate the distances
Please download the model from here: https://github.com/mrpeerat/CL-ReLKT
- Go to `models`
- Select the `xquad` folder
- Download the `student_best_supported_languages.zip` folder as a zip file

In [8]:
# !unzip student_best_supported_languages.zip

In [9]:
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

In [10]:
# define global cache dictionaries
cache_whole_texts = {}
cache_batches = {}

def encode_in_batch(model, texts):
    # cache for whole texts
    texts_tuple = tuple(texts) # since lists can't be dict keys
    if texts_tuple in cache_whole_texts:
        return cache_whole_texts[texts_tuple]

    batch_size = 16
    all_embeddings = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        # cache for batches
        batch_tuple = tuple(batch_texts)
        if batch_tuple in cache_batches:
            embeddings = cache_batches[batch_tuple]
        else:
            embeddings = cache_individual_texts(model, batch_texts)
            cache_batches[batch_tuple] = embeddings
        all_embeddings.append(embeddings)
    
    all_embeddings = tf.concat(all_embeddings, axis=0).numpy()

    # save to whole texts cache
    cache_whole_texts[texts_tuple] = all_embeddings

    return all_embeddings

@lru_cache(maxsize=None) # cache for individual texts
def cache_individual_texts(model, batch_texts):
    embeddings = model(batch_texts)
    return embeddings

In [11]:
# Calculate Distance
def calculate_distance(model, df, col_name):
    all_cols = df.columns

    original_questions_embed = encode_in_batch(model, df['question'].tolist())

    # If the column has an english counterpart perform harmonic distance
    if col_name.replace('th_', 'en_') in all_cols:
        th_embed = encode_in_batch(model, df[col_name].tolist())
        en_embed = encode_in_batch(model, df[col_name.replace('th_', 'en_')].tolist())
    
        # Compute the cosine distance
        dis_to_en = []
        dis_to_th = []
        for a, b in zip(original_questions_embed, th_embed):
            dis_to_th.append(cosine(a, b))

        for a, b in zip(original_questions_embed, en_embed):
            dis_to_en.append(cosine(a, b))
        
        # Harmonic distance
        # https://en.wikipedia.org/wiki/Harmonic_mean
        return (2 * dis_to_en * dis_to_th) / (dis_to_en + dis_to_th) 
    else:
        # If the column has no english counterpart perform cosine distance
        embed = encode_in_batch(model, df[col_name].tolist())
        dis_to_th = []
        for a, b in zip(original_questions_embed, embed):
            dis_to_th.append(cosine(a, b))

        return dis_to_th

In [12]:
all_augment_cols = [col for col in data.columns if col.startswith('th_')]
all_augment_cols

['th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_ltw2v_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_thai2fit_aug',
 'th_wordnet_aug']